## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN","NAME"], axis=1, inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# View the application_df withthe adjustmenst we have made so far.
application_df = pd.DataFrame(application_df).reindex()
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Look at APPLICATION_TYPE value counts for binning creating an exploartory data frame.
application_exp_df = application_df['APPLICATION_TYPE'].value_counts().reindex()
application_exp_df = pd.DataFrame(application_exp_df)
application_exp_df

,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156
T13,66


In [6]:
# Choose a cutoff value and view the selected 'APPLICATION_TYPE'.
application_exp_df = application_exp_df.loc[application_exp_df['APPLICATION_TYPE']<500]
application_exp_df

,APPLICATION_TYPE
T9,156
T13,66
T12,27
T2,16
T25,3
T14,3
T29,2
T15,2
T17,1


In [7]:
# Create a list of the 'APPLICATION_TYPE' to be replaced.
application_types_to_replace = application_exp_df.index.to_list()
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [8]:
# Preview the data frame to be updated.
application_df = pd.DataFrame(application_df).reindex()
application_df.head(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [9]:
# Use the created variable list 'application_types_to_replace' to replace the selected cut off data frame values.
for app in application_types_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')
application_updated_df =pd.DataFrame(application_df)
application_updated_df.head(10)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [10]:
# View the 'APLLICATION_TYPE' values to observe the changes.
application_updated_list_df = application_updated_df['APPLICATION_TYPE'].value_counts().reindex()
application_updated_list_df = pd.DataFrame(application_updated_list_df)
application_updated_list_df

,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [11]:
# View the updated data frame.
application_updated_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [12]:
# Look at CLASSIFICATION value counts for binning
classification_list_df = application_updated_df['CLASSIFICATION'].value_counts().reindex()
classification_list_df = pd.DataFrame(classification_list_df)
classification_list_df

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1
C4500,1


In [13]:
# You may find it helpful to look at CLASSIFICATION value counts >1, so we will create an exploratory dataframe to view of the outcome.
classification_list_exp_df = classification_list_df.loc[classification_list_df['CLASSIFICATION']>=1]
classification_list_exp_df

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1
C4500,1


In [14]:
# Choose a cutoff value and view the selected 'CLASSIFICATION'.
classification_list_exp_df = classification_list_exp_df.loc[classification_list_exp_df['CLASSIFICATION']<1800]
classification_list_exp_df

,CLASSIFICATION
C7000,777
C1700,287
C4000,194
C5000,116
C1270,114
...,...
C4120,1
C8210,1
C2561,1
C4500,1


In [15]:
# Create a list of classifications to be replaced
# use the variable name `classifications_to_replace` to store the list.
classifications_to_replace = classification_list_exp_df.index.to_list()
classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [16]:
# Replace in dataframe
for cls in classifications_to_replace:
   application_updated_df['CLASSIFICATION'] = application_updated_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_updated_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
model_df = pd.get_dummies(application_updated_df, columns=['CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'APPLICATION_TYPE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'APPLICATION_TYPE'])
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   ASK_AMT                       34299 non-null  int64
 1   IS_SUCCESSFUL                 34299 non-null  int64
 2   CLASSIFICATION_C1000          34299 non-null  uint8
 3   CLASSIFICATION_C1200          34299 non-null  uint8
 4   CLASSIFICATION_C2000          34299 non-null  uint8
 5   CLASSIFICATION_C2100          34299 non-null  uint8
 6   CLASSIFICATION_C3000          34299 non-null  uint8
 7   CLASSIFICATION_Other          34299 non-null  uint8
 8   AFFILIATION_CompanySponsored  34299 non-null  uint8
 9   AFFILIATION_Family/Parent     34299 non-null  uint8
 10  AFFILIATION_Independent       34299 non-null  uint8
 11  AFFILIATION_National          34299 non-null  uint8
 12  AFFILIATION_Other             34299 non-null  uint8
 13  AFFILIATION_Regional          3

In [18]:
model_df

,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,SPECIAL_CONSIDERATIONS_Y,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8
0,5000,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,5000,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,6692,1,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,142590,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34295,5000,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
34296,5000,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
34297,5000,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
# Split our preprocessed data into our features and target arrays
y = model_df['IS_SUCCESSFUL'].ravel()
y[:-1]

array([1, 1, 0, ..., 0, 0, 1])

In [20]:
X = model_df.copy()
X.drop(['IS_SUCCESSFUL'], axis=1, inplace=True)
X.head(10)


,ASK_AMT,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,SPECIAL_CONSIDERATIONS_Y,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8
0,5000,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,108590,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,5000,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,6692,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
5,5000,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
6,31452,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
7,7508025,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
8,94389,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
9,5000,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [21]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
 #YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=180, activation="tanh", input_dim=53))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=150, activation="sigmoid", input_dim=53))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=200, activation="relu", input_dim=53))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 180)               9720      
                                                                 
 dense_9 (Dense)             (None, 150)               27150     
                                                                 
 dense_10 (Dense)            (None, 200)               30200     
                                                                 
 dense_11 (Dense)            (None, 1)                 201       
                                                                 
Total params: 67,271
Trainable params: 67,271
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=60)

Epoch 1/60
858/858 [==============================] - 3s 2ms/step - loss: 0.5746 - accuracy: 0.7199
Epoch 2/60
858/858 [==============================] - 3s 4ms/step - loss: 0.5596 - accuracy: 0.7283
Epoch 3/60
858/858 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7311
Epoch 4/60
858/858 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7320
Epoch 5/60
858/858 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7320
Epoch 6/60
858/858 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7329
Epoch 7/60
858/858 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7335
Epoch 8/60
858/858 [==============================] - 3s 4ms/step - loss: 0.5486 - accuracy: 0.7344
Epoch 9/60
858/858 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7347
Epoch 10/60
858/858 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7345

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5653 - accuracy: 0.7241 - 293ms/epoch - 1ms/step
Loss: 0.565319836139679, Accuracy: 0.7240524888038635


In [33]:
# Export our model to HDF5 file
nn_model.save_weights('AlphabetSoupCharity_Optimisation.hdf5')